In [ ]:
%%capture
!pip install langchain openai langchainhub duckduckgo-search wikipedia

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# **ReAct Prompting Framework Overview**

The ReAct framework was [introduced in a 2022 paper, ReAct: Synergizing Reasoning and Acting in Language Models, by researchers at Google](https://blog.research.google/2022/11/react-synergizing-reasoning-and-acting.html).

### 🌐 **ReAct (Reasoning + Acting) Explained**

- Blends reasoning with action in language models.

- Generates reasoning traces alongside task-specific actions.

### 🔗 **Capabilities of ReAct**

1. 🔄 **Interleaved Reasoning and Acting**: Fuses reasoning with actions for fluid updates.

2. 🌍 **External Interactions**: Accesses external sources like Wikipedia for more info.

3. 🧑‍🎨 **Human-like Problem Solving**: Offers easy-to-follow, logical trajectories.

4. 🛠️ **Wide-Ranging Use Cases**: Ideal for varied tasks like QA, text games, web browsing.

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2210.03629/assets/x1.png" alt="Image Description" style="width:100%">
  <figcaption>Comparison of four prompting methods, (a) Standard, (b) Chain-of-thought (CoT, Reason Only), (c) Act-only, and (d) (Reason+Act), solving a HotpotQA question. (2) Comparison of (a) Act-only and (b) prompting methods to solve an interactive AlfWorld task. In both domains, the initial prompts of in-context examples are ommitted, and only show task solving trajectories generated by the model (Act, Thought) and the environment (Obs).</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2210.03629">Image Source</a>
</figure>


### 🌟 **Why ReAct Stands Out**

1. **Balanced Emphasis on Reasoning & Acting**: Surpasses models limited to just reasoning or action.

2. **Engages with External Data**: Pulls in real-time info for better accuracy.

3. **Boosted Transparency**: Step-by-step approach clarifies decision-making.


# 🤖 **Overview of ReAct in LangChain**

- ReAct is an agent type in LangChain.

- It uses the ReAct framework to pick tools based on their descriptions.

- Useful for chatbots, document question answering, etc.

- Context-aware for better response generation.

## 🚀 **Steps to Implement an Agent**

1. ✍️ **Prompt Template Creation**: Guide the agent on tasks and output formatting.

2. 🧩 **Output Parser Definition**: Convert LLM output into actionable responses.

3. 🧠 **LLM Setup**: Choose the preferred language model.

4. 🛑 **Stop Sequence Definition**: Instruct the LLM when to stop output.

5. 🤹 **Agent Setup**: Merge LLM, prompt template, output parser, and stop sequence.

6. 🏃 **Agent Usage**: Operate the agent using an executor with user inputs.

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain import Wikipedia
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [ ]:
search = DuckDuckGoSearchRun()

search_tool = Tool(
    name="Current Event Search",
    description = "Useful for when you need to search about current events",
    func=search.run,
    return_direct=True
)

wiki = Wikipedia()

wiki_tool = Tool(
    name="Noun Search",
    description = "Useful for when you need to search for information about nouns.",
    func=wiki.search,
    return_direct=True
)

tool_belt = [search_tool, wiki_tool]

In [ ]:
# Set up the base template
template = """Answer the following questions as best you can.

 You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought: {agent_scratchpad}

"""

# 🔧 **Custom Prompt Template for LLM Agent**

- Inherits from `StringPromptTemplate`.

- `format` method key for prompt formatting.

## 🛠️ **Inside the `format` Method**

1. 📥 Receives keyword arguments (`**kwargs`).

2. 🔄 Retrieves and formats intermediate steps `(AgentAction, Observation tuples)`.

3. 🧠 Stores formatted steps in `thoughts` variable.

4. 📝 Sets `agent_scratchpad` to `thoughts`.

5. 🧩 Creates `tools` variable with tool names and descriptions.

6. 🏷️ Forms `tool_names` by joining tool names.

7. 🎨 Finalizes and returns the formatted template using keyword arguments.

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tool_belt,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

# 🔍 **Creating a Custom Output Parser for LangChain Agent**

- Inherits from `AgentOutputParser`.

- The `parse` method is key for parsing LLM output.

### 📝 **Inside the `parse` Method**

1. 📩 Takes in LLM output as a string.

2. 🔄 Decides between `AgentAction` and `AgentFinish`.

3. 🎯 Checks for "Final Answer:" to trigger `AgentFinish`.
   - Extracts final answer if present.

4. 🕵️‍♂️ If no finish trigger, uses regex to parse action and input.

5. 🚀 Extracts action from regex's first group, and input from second.

6. 📦 Returns `AgentAction` object with parsed action and input.

In [ ]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

In [ ]:
llm = OpenAI(temperature=0)

# 🛑 **Defining the Stop Sequence for LLM Responses**

- Tailored to the specific prompt and model.

- Usually aligns with the token denoting the start of an Observation.

- Prevents LLM from creating unintended observations.

### 🤖 **Setting Up the Agent**

1. 🔗 **Combine Key Components**: Merge the LLM, output parser, stop sequence.

2. 🧰 **Include Allowed Tools**: Integrate tools permissible for the agent's use.

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 🚀 **LLMSingleActionAgent in LangChain**

- Designed for single actions based on input.

- Utilizes a language model (LLM) for decision-making.

### 📝 **How It Works**

1. 📥 **Inputs**: Receives a prompt, tool list, and past steps.

2. 🤖 **LLM Processing**: Analyzes prompt, context, and conversation state.

3. 🧠 **Decision-Making**: Uses LLM to choose the next action.

4. 🛠️ **Action Generation**: Determines tool to use and its input.

5. 🎯 **Role as a Reasoning Engine**: Leverages LLM to make informed actions.

In [ ]:
tool_names = [tool.name for tool in tool_belt]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tool_belt,
    verbose=True)

agent_executor.run("Tell me about solar eclipses in 2023")



> Entering new AgentExecutor chain...
I should search for current events related to solar eclipses in 2023.

Action: Current Event Search

Action Input: solar eclipses in 2023


Observation:Based on observations from several NASA missions, the map details the path of the Moon's shadow as it crosses the contiguous U.S. during the annular solar eclipse on October 14, 2023, and total solar eclipse on April 8, 2024. Oct. 12, 2023, 1:49 p.m. ET This weekend, millions of people will feast their eyes on the latest celestial marvel: an annular solar eclipse gracing the skies of the Western Hemisphere. The... The Saturday, Oct. 14, 2023, annular solar eclipse will cross North, Central, and South America. It will be visible in parts of the United States, Mexico, and many countries in South and Central America. NASA's Eclipse Explorer is an interactive map designed to enhance your eclipse-viewing experience. Don't Miss the 2023 Annular Solar Eclipse By Anne Buckle. Published 27-Sep-2023 On Octo

"Based on observations from several NASA missions, the map details the path of the Moon's shadow as it crosses the contiguous U.S. during the annular solar eclipse on October 14, 2023, and total solar eclipse on April 8, 2024. Oct. 12, 2023, 1:49 p.m. ET This weekend, millions of people will feast their eyes on the latest celestial marvel: an annular solar eclipse gracing the skies of the Western Hemisphere. The... The Saturday, Oct. 14, 2023, annular solar eclipse will cross North, Central, and South America. It will be visible in parts of the United States, Mexico, and many countries in South and Central America. NASA's Eclipse Explorer is an interactive map designed to enhance your eclipse-viewing experience. Don't Miss the 2023 Annular Solar Eclipse By Anne Buckle. Published 27-Sep-2023 On October 14, an annular solar eclipse will sweep across North and South America. The timeanddate team will be on the ground in Roswell, New Mexico, streaming it LIVE to you. When the Moon is far f

# LangChain does all the work for you

Using the same set of tools that were defined at the start of the notebook, you can also define a ReAct agent as follows:

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# initialize agent with the tools, language model, and the type of agent
agent = initialize_agent(tool_belt,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True
                         )

In [ ]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Current Event Search: Useful for when you need to search about current events
Noun Search: Useful for when you need to search for information about nouns.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Current Event Search, Noun Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
agent.run("Tell me about solar eclipses in 2023")



> Entering new AgentExecutor chain...
 I need to find out information about solar eclipses in 2023
Action: Current Event Search
Action Input: Solar eclipses in 2023
Observation: Oct. 12, 2023, 1:49 p.m. ET This weekend, millions of people will feast their eyes on the latest celestial marvel: an annular solar eclipse gracing the skies of the Western Hemisphere. The... The Saturday, Oct. 14, 2023, annular solar eclipse will cross North, Central, and South America. It will be visible in parts of the United States, Mexico, and many countries in South and Central America. NASA's Eclipse Explorer is an interactive map designed to enhance your eclipse-viewing experience. Based on observations from several NASA missions, the map details the path of the Moon's shadow as it crosses the contiguous U.S. during the annular solar eclipse on October 14, 2023, and total solar eclipse on April 8, 2024. Watching a solar eclipse is a human experience, one in which people across the world and across tim

"Oct. 12, 2023, 1:49 p.m. ET This weekend, millions of people will feast their eyes on the latest celestial marvel: an annular solar eclipse gracing the skies of the Western Hemisphere. The... The Saturday, Oct. 14, 2023, annular solar eclipse will cross North, Central, and South America. It will be visible in parts of the United States, Mexico, and many countries in South and Central America. NASA's Eclipse Explorer is an interactive map designed to enhance your eclipse-viewing experience. Based on observations from several NASA missions, the map details the path of the Moon's shadow as it crosses the contiguous U.S. during the annular solar eclipse on October 14, 2023, and total solar eclipse on April 8, 2024. Watching a solar eclipse is a human experience, one in which people across the world and across time can share a moment of cosmic wonder, writes Don Lincoln. ... 2023 Link Copied! Ad Feedback ... USA TODAY Millions of Americans could be able to see a spectacular solar eclipse o